Primero importamos ambos datas y les echamos un vistazo. Importo ambos porque desde el mismo jupyter haré todo el proceso.

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet, Lars, BayesianRidge, ARDRegression, TweedieRegressor
from sklearn.linear_model import TheilSenRegressor, HuberRegressor, RANSACRegressor


In [10]:
data= pd.read_csv("../Data/train.csv")
datat=pd.read_csv ("../Data/test.csv")

In [275]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40455 non-null  float64
 1   cut      40455 non-null  int64  
 2   color    40455 non-null  int64  
 3   clarity  40455 non-null  float64
 4   depth    40455 non-null  float64
 5   table    40455 non-null  float64
 6   x        40455 non-null  float64
 7   y        40455 non-null  float64
 8   z        40455 non-null  float64
 9   price    40455 non-null  float64
dtypes: float64(8), int64(2)
memory usage: 3.1 MB


Tenemos 11 columnas, 6 numéricas, 3 categóricas, un target y el id:
* x, y, z, carat, table y depth son las numéricas y aparentemente no hay que hacerles nada. (6)
* [cut](https://www.loosediamondsreviews.com/diamondcut.html): tenemos que categorizarla entre 1 y 5 de forma escalar, ya que mide la calidad del corte.(7)
* [color](https://es.wikipedia.org/wiki/Color_en_los_diamantes#:~:text=Los%20diamantes%20presentan%20s%C3%B3lo%20un,%2C%20marr%C3%B3n%2C%20violeta%20y%20negro.): tenemos que categorizarla entre las D y la Z, estén o no presentes en el data, entre 2 letras hay un salto de 2 * calidades. Por otra parte sería bueno echarle un vistazo a las "Z"s, que si bien a priori son las más baratas, puede haber excepciones. (8)
* [clarity](https://www.tiffany.com.mx/engagement/the-tiffany-guide-to-diamonds/clarity/#:~:text=La%20claridad%20es%20una%20medida,manchas%20visibles%20(imperfecciones%20externas).): categorizable de peor a mejor. (9)
* Tambien vamos a elminar la id y, finalmente estableceremos price como target. 

Creamos una función con todo lo que le vamos a hacer, y la aplicamos:

In [12]:
categorizacion (data)
categorizacion (datat)

,carat,cut,color,clarity,depth,table,x,y,z
0,2.01,5,3,2.0,61.9,57.0,8.14,8.05,5.01
1,0.49,2,7,3.0,57.5,60.0,5.18,5.25,3.00
2,1.03,4,5,2.0,58.6,62.0,6.65,6.60,3.88
3,0.90,3,6,2.0,63.0,56.0,6.11,6.15,3.86
4,0.59,5,7,2.0,62.5,55.0,5.35,5.40,3.36
...,...,...,...,...,...,...,...,...,...
13480,0.41,2,5,1.0,63.8,57.0,4.72,4.69,3.00
13481,2.05,3,3,2.5,63.5,62.0,8.05,7.91,5.07
13482,2.22,4,2,2.5,60.5,59.0,8.41,8.37,5.08
13483,0.51,5,6,1.5,58.3,62.0,5.20,5.19,3.04


In [449]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40455 non-null  float64
 1   cut      0 non-null      float64
 2   color    0 non-null      float64
 3   clarity  0 non-null      float64
 4   depth    40455 non-null  float64
 5   table    40455 non-null  float64
 6   x        40455 non-null  float64
 7   y        40455 non-null  float64
 8   z        40455 non-null  float64
 9   price    40455 non-null  float64
dtypes: float64(10)
memory usage: 3.1 MB


Una vex hecho esto, vamos a ver las correlaciones, en un principio y por lo que he leido, hay varias variables que tienen que ver con el tamaño y que estarán correlacionadas entre si

In [20]:
data.corr()

,carat,cut,color,clarity,depth,table,x,y,z,price
carat,1.000000,-0.136512,-0.295110,-0.347222,0.029747,0.182453,0.974714,0.944353,0.948460,0.920292
cut,-0.136512,1.000000,0.024676,0.187792,-0.211912,-0.438306,-0.128336,-0.123047,-0.150653,-0.090436
color,-0.295110,0.024676,1.000000,-0.025310,-0.048786,-0.028571,-0.273433,-0.264744,-0.270077,-0.158279
clarity,-0.347222,0.187792,-0.025310,1.000000,-0.064298,-0.164575,-0.366545,-0.350738,-0.359719,-0.210824
depth,0.029747,-0.211912,-0.048786,-0.064298,1.000000,-0.291668,-0.023705,-0.028183,0.095489,0.003437
table,0.182453,-0.438306,-0.028571,-0.164575,-0.291668,1.000000,0.197420,0.184191,0.152496,0.159969
x,0.974714,-0.128336,-0.273433,-0.366545,-0.023705,0.197420,1.000000,0.967309,0.966396,0.957815
y,0.944353,-0.123047,-0.264744,-0.350738,-0.028183,0.184191,0.967309,1.000000,0.941970,0.928896
z,0.948460,-0.150653,-0.270077,-0.359719,0.095489,0.152496,0.966396,0.941970,1.000000,0.930774
price,0.920292,-0.090436,-0.158279,-0.210824,0.003437,0.159969,0.957815,0.928896,0.930774,1.000000


Por lo que hemos leido, carat incluye x, y y z, por lo que quizás serían prescindibles. En cualquier caso, quiero hacer un modelo rapido para hacer una submission y ver que sé hacerla. Así que vamos a sacar el primer modelo a lo bruto sin hacer mucho más:

In [24]:
columnas_x = [a for a in (list(data.columns)) if a != "price"]
X = data[columnas_x]
y= data.price

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [25]:
samplelr=LinearRegression ()

In [26]:
samplelr.fit(X_train, y_train)

LinearRegression()

In [27]:
y_pred= samplelr.predict (X_train)

In [28]:
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_train,y_pred)))

RMSE:  0.18198814114879042


In [29]:
submission= samplelr.predict (datat)

In [30]:
submission

array([9.74252551, 7.36921785, 8.57124362, ..., 9.87656414, 7.09163483,
       7.25821335])

In [31]:
datat["price"] = samplelr.predict (datat)

In [32]:
submis= datat[["price"]]

In [33]:
submis.index.rename ("id", inplace=True)

In [35]:
submis.to_csv("../Data/subprueba.csv")

Una vez hecho esto, que era por ver si iba a tener problemas, vamos a tomarnoslo un poco más en serio. 


Primero voy a aplicar varios modelos a nuestro data.

In [67]:
models = { "ridge": Ridge(),
          "lasso": Lasso(),
          "sgd": SGDRegressor(),
          "knn": KNeighborsRegressor(),
          "gradient": GradientBoostingRegressor(),
          "linear": LinearRegression ()
}

In [443]:
probando_modelos(models)

--------ridge--------
MSE:  0.05146178664329969
--------lasso--------
MSE:  0.8769753489338427
--------sgd--------
MSE:  1488128980634365.2
--------knn--------
MSE:  0.028537108100358425
--------gradient--------
MSE:  0.012173781248689524
--------linear--------
MSE:  0.05150747922908061


Parece que el unico interesante es gradient, que mejora mucho a los demás. Voy a probarla con X_test. 

In [54]:
gradient_model=  GradientBoostingRegressor ()

In [55]:
gradient_model.fit (X_train, y_train)

GradientBoostingRegressor()

In [57]:
y_pred1=gradient_model.predict (X_test)

In [59]:
print("MSE: ", metrics.mean_squared_error(y_test,y_pred1))

MSE:  0.011619751949012217


No varia prácticamente nada de las que hemos probado, eso está bien porque nos indica que nuestra mse ya nos incluye el posible overfitting. 

Ahora vamos a buscar los parámetros ideales:

In [450]:
# no se porqué en este momento supuse que gridsearch solo servia con decisiontree, por lo que hice la búsqueda de parámetros un
# poco a lo bruto

In [68]:
gmodels = { "g1": GradientBoostingRegressor(n_estimators=200),
          "g2": GradientBoostingRegressor(n_estimators=200, learning_rate=0.15),
          "g3": GradientBoostingRegressor(n_estimators=200, criterion="mse"),
          "g4": GradientBoostingRegressor(n_estimators=300),
          "g5": GradientBoostingRegressor(n_estimators=300, criterion="mse"),
          "g6": GradientBoostingRegressor(n_estimators=300, learning_rate=0.2)
}

In [445]:
probando_modelos_cross (gmodels)

g1 MSE -0.009438632327618813
g2 MSE -0.008914322631818771
g3 MSE -0.009452287088027223
g4 MSE -0.008794111025934359
g5 MSE -0.00880213531722418
g6 MSE -0.008299880394818382


Se observa qué:
* las 6 mejoran nuestro modelo inicial.
* El número de estimadores en 200 es mejor que en 100, en 300 mejor que en 200, pero puede overfitear.
* indicarle mse en "criterion" mejora los resultados, si buscamos mejorar el mse, parace lógico decirle que lo utilice en el algoritmo
* Learning rate también mejora los resultados, compensa de alguna manera el numero de estimadores.

In [90]:
gmodels2 = { "g7": GradientBoostingRegressor(n_estimators=500, learning_rate=0.2, criterion="mse"),
          "g8": GradientBoostingRegressor(n_estimators=400, learning_rate=0.2, criterion="mse"),
          "g9": GradientBoostingRegressor(n_estimators=400, learning_rate=0.3, criterion="mse"),
          "g10" : GradientBoostingRegressor(n_estimators=300, learning_rate=0.3, criterion="mse"),
          "g11" : GradientBoostingRegressor(n_estimators=500, learning_rate=0.4, criterion="mse"),
          "g12": GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, criterion="mse"),
          "g13" : GradientBoostingRegressor(n_estimators=300, learning_rate=0.2, criterion="mse"),
          "g14": GradientBoostingRegressor(n_estimators=500, learning_rate=0.3, criterion="mse"),
          "g15": GradientBoostingRegressor(n_estimators=500, learning_rate=0.5, criterion="mse")

}

In [447]:
probando_modelos_cross (gmodels2)

g7 MSE -0.0079556970989251
g8 MSE -0.008077587644358463
g9 MSE -0.008071634080564624
g10 MSE -0.00823990090172973
g11 MSE -0.007960910363890591
g12 MSE -0.008787497765073003
g13 MSE -0.008278302511492865
g14 MSE -0.007952864983275017
g15 MSE -0.008167333169062086


Seguimos ajustando, a falta de probar con nuestro test. Podemos ver:
* se ajustan las diferencias, pero 500 mejor que 400
* si abrimos demasiado el learning_rate comienza a disminuir el resultado

Vamos con la ultima prueba, el número de estimadores lo voy a exagerar. No creo que me dé buenos resultados, pero quiero ver que sale

In [93]:
gmodels3 = { "g16": GradientBoostingRegressor(n_estimators=500, learning_rate=0.1, criterion="mse"),
          "g17": GradientBoostingRegressor(n_estimators=700, learning_rate=0.1, criterion="mse"),
          "g18": GradientBoostingRegressor(n_estimators=700, learning_rate=0.2, criterion="mse"),
          "g19" : GradientBoostingRegressor(n_estimators=700, learning_rate=0.3, criterion="mse"),
          "g20" : GradientBoostingRegressor(n_estimators=700, learning_rate=0.4, criterion="mse"),
          "g21": GradientBoostingRegressor(n_estimators=700, learning_rate=0.5, criterion="mse"),
          "g22" : GradientBoostingRegressor(n_estimators=700, learning_rate=0.6, criterion="mse"),
          "g23": GradientBoostingRegressor(n_estimators=1000, learning_rate=0.3, criterion="mse"),
          "g24": GradientBoostingRegressor(n_estimators=1000, learning_rate=0.5, criterion="mse"),
          "g25": GradientBoostingRegressor(n_estimators=1000, learning_rate=0.7, criterion="mse"),
          "g26": GradientBoostingRegressor(n_estimators=1500, learning_rate=0.3, criterion="mse"),
          "g26": GradientBoostingRegressor(n_estimators=1500, learning_rate=0.7, criterion="mse")

}

In [96]:
probando_modelos_cross (gmodels3)

g16 MSE -0.008547841914336485
g17 MSE -0.008298851170234116
g18 MSE -0.008108877221806112
g19 MSE -0.008178944911612376
g20 MSE -0.008481344932443124
g21 MSE -0.008390901164400461
g22 MSE -0.008772336356117785
g23 MSE -0.008144235702112963
g24 MSE -0.008471058175072179
g25 MSE -0.009605196004309872
g26 MSE -0.009890697149181667


Y ahora haré de nuevo otro testeo, incluyendo los modelos que mejores resultados han sacado, y bajando el learning_rate para los estimadores altísimos que he puesto en la anterior prueba.

In [97]:
gmodels4 = { 
    
          "g7": GradientBoostingRegressor(n_estimators=500, learning_rate=0.2, criterion="mse"),
          "g14" : GradientBoostingRegressor(n_estimators=500, learning_rate=0.3, criterion="mse"),
          "g14b" : GradientBoostingRegressor(n_estimators=500, learning_rate=0.25, criterion="mse"),
          "g18b" : GradientBoostingRegressor(n_estimators=700, learning_rate=0.25, criterion="mse"),
          "g17": GradientBoostingRegressor(n_estimators=700, learning_rate=0.1, criterion="mse"),
          "g18": GradientBoostingRegressor(n_estimators=700, learning_rate=0.2, criterion="mse"),
          "g19": GradientBoostingRegressor(n_estimators=700, learning_rate=0.3, criterion="mse"),
          "g23" : GradientBoostingRegressor(n_estimators=1000, learning_rate=0.3, criterion="mse"),
          "g26" : GradientBoostingRegressor(n_estimators=100, learning_rate=0.2, criterion="mse"),
          "g27" : GradientBoostingRegressor(n_estimators=1500, learning_rate=0.3, criterion="mse"),
          "g28": GradientBoostingRegressor(n_estimators=1500, learning_rate=0.2, criterion="mse"),
          "g29": GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, criterion="mse")

}

In [99]:
probando_modelos_cross (gmodels4)

g7 MSE -0.008253128849772527
g14 MSE -0.008293769074612187
g14b MSE -0.008178039788034039
g18b MSE -0.008027390003129662
g17 MSE -0.008311653987278681
g18 MSE -0.00811744445312339
g19 MSE -0.008194918386221153
g23 MSE -0.00816198581737589
g26 MSE -0.01016655847478131
g27 MSE -0.008227193548511945
g28 MSE -0.007999097391703213
g29 MSE -0.008098853330303284


In [100]:
gmodels5 = { 
    
          "g30": GradientBoostingRegressor(n_estimators=1500, learning_rate=0.25, criterion="mse"),
          "g31" : GradientBoostingRegressor(n_estimators=1500, learning_rate=0.27, criterion="mse"),
          "g32" : GradientBoostingRegressor(n_estimators=2000, learning_rate=0.25, criterion="mse"),
          "g18c" : GradientBoostingRegressor(n_estimators=700, learning_rate=0.27, criterion="mse"),
          "g33": GradientBoostingRegressor(n_estimators=2000, learning_rate=0.27, criterion="mse"),
          "g34": GradientBoostingRegressor(n_estimators=2500, learning_rate=0.27, criterion="mse"),
          "g35": GradientBoostingRegressor(n_estimators=2500, learning_rate=0.25, criterion="mse"),
          "g36" : GradientBoostingRegressor(n_estimators=3000, learning_rate=0.25, criterion="mse"),
          "g37" : GradientBoostingRegressor(n_estimators=4000, learning_rate=0.25, criterion="mse"),
          "g38" : GradientBoostingRegressor(n_estimators=5000, learning_rate=0.25, criterion="mse"),
          "g39": GradientBoostingRegressor(n_estimators=6000, learning_rate=0.25, criterion="mse"),
          "g29": GradientBoostingRegressor(n_estimators=7000, learning_rate=0.25, criterion="mse")

}

In [102]:
probando_modelos_cross (gmodels5)

g30 MSE -0.008058277300711155
g31 MSE -0.008108957853244276
g32 MSE -0.00806518125742603
g18c MSE -0.008156923590402007
g33 MSE -0.008181984479487339
g34 MSE -0.008279435940388241
g35 MSE -0.008183564169390655
g36 MSE -0.00822720460123322
g37 MSE -0.008413828834196205
g38 MSE -0.008585062432874671
g39 MSE -0.00869012909611622
g29 MSE -0.008742503417191322


In [104]:
gmodels6 = { 
    
          "g61": GradientBoostingRegressor(n_estimators=700, learning_rate=0.25, criterion="mse"),
          "g62" : GradientBoostingRegressor(n_estimators=800, learning_rate=0.25, criterion="mse"),
          "g63" : GradientBoostingRegressor(n_estimators=900, learning_rate=0.25, criterion="mse"),
          "g64" : GradientBoostingRegressor(n_estimators=1000, learning_rate=0.25, criterion="mse"),
          "g65": GradientBoostingRegressor(n_estimators=1100, learning_rate=0.27, criterion="mse"),
          "g66": GradientBoostingRegressor(n_estimators=1200, learning_rate=0.27, criterion="mse"),
          "g67": GradientBoostingRegressor(n_estimators=1300, learning_rate=0.25, criterion="mse"),
          "g68" : GradientBoostingRegressor(n_estimators=1400, learning_rate=0.25, criterion="mse"),
          "g69" : GradientBoostingRegressor(n_estimators=1500, learning_rate=0.25, criterion="mse"),
          "g70" : GradientBoostingRegressor(n_estimators=1600, learning_rate=0.25, criterion="mse"),
          "g71": GradientBoostingRegressor(n_estimators=1700, learning_rate=0.25, criterion="mse"),
          "g72": GradientBoostingRegressor(n_estimators=1800, learning_rate=0.25, criterion="mse"),
          "g73" : GradientBoostingRegressor(n_estimators=1900, learning_rate=0.25, criterion="mse"),
          "g74": GradientBoostingRegressor(n_estimators=2000, learning_rate=0.25, criterion="mse"),
          "g75": GradientBoostingRegressor(n_estimators=2100, learning_rate=0.25, criterion="mse"),
          "g76": GradientBoostingRegressor(n_estimators=2200, learning_rate=0.25, criterion="mse"),
          "g77": GradientBoostingRegressor(n_estimators=2300, learning_rate=0.25, criterion="mse")

}

In [105]:
probando_modelos_cross (gmodels6)

g61 MSE -0.008041955807486244
g62 MSE -0.00800012603715632
g63 MSE -0.00800404618669956
g64 MSE -0.00800687222726994
g65 MSE -0.008058656957863666
g66 MSE -0.008089482790428046
g67 MSE -0.008016175129264044
g68 MSE -0.008015488362341257
g69 MSE -0.008049929649960898
g70 MSE -0.008051859463429432
g71 MSE -0.008054547007559884
g72 MSE -0.008067886283408453
g73 MSE -0.008057460825560164
g74 MSE -0.008109291860586515
g75 MSE -0.008129350652664843
g76 MSE -0.00813032127484198
g77 MSE -0.008159711506248326


Con 800 estimadores saca la mejor puntuación, de todos modos haré una submission con 800 y otra con 1000, y después seguiremos trabajando en otros. 

En este punto tengo bastante confusión con los x_train, test, dataframes... así que voy a reimportar los datas y entrenarlos directamente con nuestro train:

In [111]:
data= pd.read_csv("../Data/train.csv")
datat=pd.read_csv ("../Data/test.csv")

In [112]:
categorizacion (data)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.01,4,4,3.0,62.7,58.0,6.36,6.31,3.97,8.754
1,0.60,2,4,2.5,64.3,57.0,5.31,5.38,3.43,7.492
2,1.60,4,3,1.5,59.4,59.0,7.55,7.60,4.50,9.010
3,0.30,3,1,3.0,63.5,58.0,4.27,4.24,2.70,6.227
4,1.00,3,5,1.5,62.8,57.0,6.37,6.44,4.02,8.314
...,...,...,...,...,...,...,...,...,...,...
40450,0.51,5,6,2.5,62.5,55.0,5.08,5.13,3.19,7.412
40451,0.80,5,3,2.5,62.1,54.0,5.96,5.99,3.71,8.016
40452,1.53,4,4,1.5,58.7,59.0,7.51,7.45,4.39,9.070
40453,1.51,5,2,2.5,61.6,54.0,7.38,7.42,4.57,9.199


In [113]:
categorizacion (datat)

,carat,cut,color,clarity,depth,table,x,y,z
0,2.01,5,3,2.0,61.9,57.0,8.14,8.05,5.01
1,0.49,2,7,3.0,57.5,60.0,5.18,5.25,3.00
2,1.03,4,5,2.0,58.6,62.0,6.65,6.60,3.88
3,0.90,3,6,2.0,63.0,56.0,6.11,6.15,3.86
4,0.59,5,7,2.0,62.5,55.0,5.35,5.40,3.36
...,...,...,...,...,...,...,...,...,...
13480,0.41,2,5,1.0,63.8,57.0,4.72,4.69,3.00
13481,2.05,3,3,2.5,63.5,62.0,8.05,7.91,5.07
13482,2.22,4,2,2.5,60.5,59.0,8.41,8.37,5.08
13483,0.51,5,6,1.5,58.3,62.0,5.20,5.19,3.04


In [114]:
columnas_x = [a for a in (list(data.columns)) if a != "price"]
columnas_x
X = data[columnas_x]
y= data.price

Para entrenarlo con todos los datos, no haré splits. Con esto no tendré una idea inicial de cómo predigo el test, ni del posible overfitting. Pero para eso ya hemos probado el model unas cuantas veces más arriba.

In [115]:
gradient_model2 = GradientBoostingRegressor (n_estimators=800, learning_rate=0.25, criterion="mse")

In [116]:
gradient_model2.fit (X, y)

GradientBoostingRegressor(criterion='mse', learning_rate=0.25, n_estimators=800)

In [119]:
y_pred=gradient_model2.predict (datat)

In [121]:
datat["price"] = y_pred

In [122]:
submision=datat[["price"]]

In [124]:
submision.index.rename ("id", inplace=True)

In [126]:
submision.to_csv ("../Data/gradientdatacompleto.csv")

Ahora tengo que reimportar los datas, porque he modificado el datat

In [130]:
data= pd.read_csv("../Data/train.csv")
datat=pd.read_csv ("../Data/test.csv")

In [131]:
categorizacion (data)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.01,4,4,3.0,62.7,58.0,6.36,6.31,3.97,8.754
1,0.60,2,4,2.5,64.3,57.0,5.31,5.38,3.43,7.492
2,1.60,4,3,1.5,59.4,59.0,7.55,7.60,4.50,9.010
3,0.30,3,1,3.0,63.5,58.0,4.27,4.24,2.70,6.227
4,1.00,3,5,1.5,62.8,57.0,6.37,6.44,4.02,8.314
...,...,...,...,...,...,...,...,...,...,...
40450,0.51,5,6,2.5,62.5,55.0,5.08,5.13,3.19,7.412
40451,0.80,5,3,2.5,62.1,54.0,5.96,5.99,3.71,8.016
40452,1.53,4,4,1.5,58.7,59.0,7.51,7.45,4.39,9.070
40453,1.51,5,2,2.5,61.6,54.0,7.38,7.42,4.57,9.199


In [132]:
categorizacion (datat)

,carat,cut,color,clarity,depth,table,x,y,z
0,2.01,5,3,2.0,61.9,57.0,8.14,8.05,5.01
1,0.49,2,7,3.0,57.5,60.0,5.18,5.25,3.00
2,1.03,4,5,2.0,58.6,62.0,6.65,6.60,3.88
3,0.90,3,6,2.0,63.0,56.0,6.11,6.15,3.86
4,0.59,5,7,2.0,62.5,55.0,5.35,5.40,3.36
...,...,...,...,...,...,...,...,...,...
13480,0.41,2,5,1.0,63.8,57.0,4.72,4.69,3.00
13481,2.05,3,3,2.5,63.5,62.0,8.05,7.91,5.07
13482,2.22,4,2,2.5,60.5,59.0,8.41,8.37,5.08
13483,0.51,5,6,1.5,58.3,62.0,5.20,5.19,3.04


In [133]:
gradient_model3 = GradientBoostingRegressor (n_estimators=1000, learning_rate=0.25, criterion="mse")

In [134]:
gradient_model2.fit (X, y)

GradientBoostingRegressor(criterion='mse', learning_rate=0.25, n_estimators=800)

In [135]:
y_pred=gradient_model2.predict (datat)

In [139]:
datat["price"] = y_pred
submision=datat[["price"]]
submision.index.rename ("id", inplace=True)

In [140]:
submision.to_csv ("../Data/gradientdatacompleto2.csv")

Esta última no ha mejorado a la primera con 800 estimadores.

Ahora vamos a probar con DecisionTree, esperando obtener algunas mejoras.

In [162]:
gs = GridSearchCV(
    estimator=DecisionTreeRegressor(),
    param_grid={
        "max_depth": [5,10,15,20, 25],
        "min_samples_split": [2,5,7,10,15,25],
        "max_features": [8, 9]
    },
    cv=5,
    verbose=3,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

In [163]:
%%time
gs.fit (X_train, y_train )# mal aplicado, tenían que ser X e y

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] max_depth=5, max_features=8, min_samples_split=2 ................
[CV]  max_depth=5, max_features=8, min_samples_split=2, score=(train=-0.039, test=-0.040), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=2 ................
[CV]  max_depth=5, max_features=8, min_samples_split=2, score=(train=-0.040, test=-0.041), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=2 ................
[CV]  max_depth=5, max_features=8, min_samples_split=2, score=(train=-0.039, test=-0.041), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=2 ................
[CV]  max_depth=5, max_features=8, min_samples_split=2, score=(train=-0.040, test=-0.039), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=2 ................
[CV]  max_depth=5, max_features=8, min_samples_split=2, score=(train=-0.040, test=-0.041), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=5 ....

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  max_depth=5, max_features=8, min_samples_split=5, score=(train=-0.040, test=-0.042), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=5 ................
[CV]  max_depth=5, max_features=8, min_samples_split=5, score=(train=-0.040, test=-0.042), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=5 ................
[CV]  max_depth=5, max_features=8, min_samples_split=5, score=(train=-0.039, test=-0.040), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=5 ................
[CV]  max_depth=5, max_features=8, min_samples_split=5, score=(train=-0.041, test=-0.040), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=5 ................
[CV]  max_depth=5, max_features=8, min_samples_split=5, score=(train=-0.039, test=-0.042), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=7 ................
[CV]  max_depth=5, max_features=8, min_samples_split=7, score=(train=-0.040, test=-0.043), total=   0.0s
[CV] max_depth=

[CV]  max_depth=5, max_features=9, min_samples_split=25, score=(train=-0.039, test=-0.041), total=   0.0s
[CV] max_depth=5, max_features=9, min_samples_split=25 ...............
[CV]  max_depth=5, max_features=9, min_samples_split=25, score=(train=-0.040, test=-0.042), total=   0.0s
[CV] max_depth=5, max_features=9, min_samples_split=25 ...............
[CV]  max_depth=5, max_features=9, min_samples_split=25, score=(train=-0.038, test=-0.040), total=   0.0s
[CV] max_depth=5, max_features=9, min_samples_split=25 ...............
[CV]  max_depth=5, max_features=9, min_samples_split=25, score=(train=-0.040, test=-0.039), total=   0.0s
[CV] max_depth=5, max_features=9, min_samples_split=25 ...............
[CV]  max_depth=5, max_features=9, min_samples_split=25, score=(train=-0.039, test=-0.042), total=   0.0s
[CV] max_depth=10, max_features=8, min_samples_split=2 ...............
[CV]  max_depth=10, max_features=8, min_samples_split=2, score=(train=-0.010, test=-0.016), total=   0.1s
[CV] max_

[CV]  max_depth=10, max_features=9, min_samples_split=7, score=(train=-0.011, test=-0.015), total=   0.1s
[CV] max_depth=10, max_features=9, min_samples_split=7 ...............
[CV]  max_depth=10, max_features=9, min_samples_split=7, score=(train=-0.010, test=-0.015), total=   0.1s
[CV] max_depth=10, max_features=9, min_samples_split=10 ..............
[CV]  max_depth=10, max_features=9, min_samples_split=10, score=(train=-0.010, test=-0.016), total=   0.1s
[CV] max_depth=10, max_features=9, min_samples_split=10 ..............
[CV]  max_depth=10, max_features=9, min_samples_split=10, score=(train=-0.011, test=-0.015), total=   0.1s
[CV] max_depth=10, max_features=9, min_samples_split=10 ..............
[CV]  max_depth=10, max_features=9, min_samples_split=10, score=(train=-0.011, test=-0.015), total=   0.1s
[CV] max_depth=10, max_features=9, min_samples_split=10 ..............
[CV]  max_depth=10, max_features=9, min_samples_split=10, score=(train=-0.011, test=-0.015), total=   0.1s
[CV] 

...

[CV]  max_depth=20, max_features=8, min_samples_split=15, score=(train=-0.005, test=-0.015), total=   0.1s
[CV] max_depth=20, max_features=8, min_samples_split=15 ..............
[CV]  max_depth=20, max_features=8, min_samples_split=15, score=(train=-0.005, test=-0.014), total=   0.1s
[CV] max_depth=20, max_features=8, min_samples_split=15 ..............
[CV]  max_depth=20, max_features=8, min_samples_split=15, score=(train=-0.005, test=-0.013), total=   0.1s
[CV] max_depth=20, max_features=8, min_samples_split=15 ..............
[CV]  max_depth=20, max_features=8, min_samples_split=15, score=(train=-0.005, test=-0.014), total=   0.1s
[CV] max_depth=20, max_features=8, min_samples_split=15 ..............
[CV]  max_depth=20, max_features=8, min_samples_split=15, score=(train=-0.004, test=-0.013), total=   0.1s
[CV] max_depth=20, max_features=8, min_samples_split=25 ..............
[CV]  max_depth=20, max_features=8, min_samples_split=25, score=(train=-0.006, test=-0.014), total=   0.1s
[CV

[CV]  max_depth=25, max_features=8, min_samples_split=5, score=(train=-0.001, test=-0.015), total=   0.1s
[CV] max_depth=25, max_features=8, min_samples_split=5 ...............
[CV]  max_depth=25, max_features=8, min_samples_split=5, score=(train=-0.001, test=-0.015), total=   0.1s
[CV] max_depth=25, max_features=8, min_samples_split=7 ...............
[CV]  max_depth=25, max_features=8, min_samples_split=7, score=(train=-0.002, test=-0.015), total=   0.1s
[CV] max_depth=25, max_features=8, min_samples_split=7 ...............
[CV]  max_depth=25, max_features=8, min_samples_split=7, score=(train=-0.002, test=-0.016), total=   0.1s
[CV] max_depth=25, max_features=8, min_samples_split=7 ...............
[CV]  max_depth=25, max_features=8, min_samples_split=7, score=(train=-0.002, test=-0.015), total=   0.1s
[CV] max_depth=25, max_features=8, min_samples_split=7 ...............
[CV]  max_depth=25, max_features=8, min_samples_split=7, score=(train=-0.002, test=-0.016), total=   0.1s
[CV] max_

[CV]  max_depth=25, max_features=9, min_samples_split=25, score=(train=-0.006, test=-0.013), total=   0.1s
[CV] max_depth=25, max_features=9, min_samples_split=25 ..............
[CV]  max_depth=25, max_features=9, min_samples_split=25, score=(train=-0.006, test=-0.013), total=   0.1s
[CV] max_depth=25, max_features=9, min_samples_split=25 ..............
[CV]  max_depth=25, max_features=9, min_samples_split=25, score=(train=-0.006, test=-0.013), total=   0.1s
[CV] max_depth=25, max_features=9, min_samples_split=25 ..............
[CV]  max_depth=25, max_features=9, min_samples_split=25, score=(train=-0.006, test=-0.013), total=   0.1s
Wall time: 25.5 s


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   25.3s finished


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [5, 10, 15, 20, 25],
                         'max_features': [8, 9],
                         'min_samples_split': [2, 5, 7, 10, 15, 25]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=3)

In [452]:
resultados_gridDT (gs)

,param_max_depth,param_max_features,param_min_samples_split,mean_test_score,mean_train_score
35,15,9,25,-0.013095,-0.006485
47,20,9,25,-0.013122,-0.006166
59,25,9,25,-0.013191,-0.006155
34,15,9,15,-0.013283,-0.005022
29,15,8,25,-0.013306,-0.006657


In [203]:
gs2 = GridSearchCV(
    estimator=DecisionTreeRegressor(),
    param_grid={
        "max_depth": [3,5,10,15,20, 25],
        "min_samples_split": [10,15,25],
        "max_features": [8, 9],
        "min_weight_fraction_leaf" : [0.1,0.2,0.05]
    },
    cv=5,
    verbose=3,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

In [204]:
gs2.fit (X_train, y_train ) #mal aplicado, tenía que ser X e y

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=-0.080, test=-0.080), total=   0.0s
[CV] max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=-0.088, test=-0.089), total=   0.0s
[CV] max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=-0.080, test=-0.080), total=   0.0s
[CV] max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=-0.087, test=-0.088), total=   0.0s
[CV] max_depth=3, max_features=8, min_samples_spl

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.2, score=(train=-0.133, test=-0.131), total=   0.0s
[CV] max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.2 
[CV]  max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.2, score=(train=-0.133, test=-0.131), total=   0.0s
[CV] max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.2 
[CV]  max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.2, score=(train=-0.133, test=-0.133), total=   0.0s
[CV] max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.05 
[CV]  max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.05, score=(train=-0.072, test=-0.072), total=   0.0s
[CV] max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.05 
[CV]  max_depth=3, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.05, score=(train=-0.074, 

[CV]  max_depth=3, max_features=9, min_samples_split=10, min_weight_fraction_leaf=0.2, score=(train=-0.132, test=-0.131), total=   0.0s
[CV] max_depth=3, max_features=9, min_samples_split=10, min_weight_fraction_leaf=0.2 
[CV]  max_depth=3, max_features=9, min_samples_split=10, min_weight_fraction_leaf=0.2, score=(train=-0.131, test=-0.136), total=   0.0s
[CV] max_depth=3, max_features=9, min_samples_split=10, min_weight_fraction_leaf=0.2 
[CV]  max_depth=3, max_features=9, min_samples_split=10, min_weight_fraction_leaf=0.2, score=(train=-0.133, test=-0.131), total=   0.0s
[CV] max_depth=3, max_features=9, min_samples_split=10, min_weight_fraction_leaf=0.2 
[CV]  max_depth=3, max_features=9, min_samples_split=10, min_weight_fraction_leaf=0.2, score=(train=-0.133, test=-0.131), total=   0.0s
[CV] max_depth=3, max_features=9, min_samples_split=10, min_weight_fraction_leaf=0.2 
[CV]  max_depth=3, max_features=9, min_samples_split=10, min_weight_fraction_leaf=0.2, score=(train=-0.133, test

[CV]  max_depth=3, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(train=-0.073, test=-0.073), total=   0.0s
[CV] max_depth=3, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05 
[CV]  max_depth=3, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(train=-0.073, test=-0.074), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=5, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=-0.069, test=-0.069), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=5, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=-0.086, test=-0.087), total=   0.0s
[CV] max_depth=5, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=5, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=-0.069, t

...

[CV]  max_depth=20, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(train=-0.056, test=-0.056), total=   0.0s
[CV] max_depth=20, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05 
[CV]  max_depth=20, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(train=-0.054, test=-0.055), total=   0.0s
[CV] max_depth=25, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=25, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=-0.076, test=-0.077), total=   0.0s
[CV] max_depth=25, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=25, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=-0.084, test=-0.086), total=   0.0s
[CV] max_depth=25, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1 
[CV]  max_depth=25, max_features=8, min_samples_split=10, min_weight_fraction_leaf=0.1, score=(train=

[CV]  max_depth=25, max_features=8, min_samples_split=25, min_weight_fraction_leaf=0.2, score=(train=-0.133, test=-0.133), total=   0.0s
[CV] max_depth=25, max_features=8, min_samples_split=25, min_weight_fraction_leaf=0.05 
[CV]  max_depth=25, max_features=8, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(train=-0.049, test=-0.049), total=   0.0s
[CV] max_depth=25, max_features=8, min_samples_split=25, min_weight_fraction_leaf=0.05 
[CV]  max_depth=25, max_features=8, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(train=-0.057, test=-0.058), total=   0.0s
[CV] max_depth=25, max_features=8, min_samples_split=25, min_weight_fraction_leaf=0.05 
[CV]  max_depth=25, max_features=8, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(train=-0.049, test=-0.049), total=   0.0s
[CV] max_depth=25, max_features=8, min_samples_split=25, min_weight_fraction_leaf=0.05 
[CV]  max_depth=25, max_features=8, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(t

[CV]  max_depth=25, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.2, score=(train=-0.133, test=-0.131), total=   0.0s
[CV] max_depth=25, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.2 
[CV]  max_depth=25, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.2, score=(train=-0.133, test=-0.133), total=   0.0s
[CV] max_depth=25, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05 
[CV]  max_depth=25, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(train=-0.049, test=-0.049), total=   0.0s
[CV] max_depth=25, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05 
[CV]  max_depth=25, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(train=-0.056, test=-0.056), total=   0.0s
[CV] max_depth=25, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05 
[CV]  max_depth=25, max_features=9, min_samples_split=25, min_weight_fraction_leaf=0.05, score=(tra

[Parallel(n_jobs=1)]: Done 540 out of 540 | elapsed:   15.8s finished


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [3, 5, 10, 15, 20, 25],
                         'max_features': [8, 9],
                         'min_samples_split': [10, 15, 25],
                         'min_weight_fraction_leaf': [0.1, 0.2, 0.05]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=3)

In [454]:
resultados_gridDT (gs2)

,param_max_depth,param_max_features,param_min_samples_split,mean_test_score,mean_train_score
95,25,8,15,-0.051888,-0.051763
98,25,8,25,-0.052199,-0.051812
44,10,8,25,-0.052378,-0.051948
38,10,8,10,-0.052582,-0.051859
62,15,8,25,-0.052669,-0.052558


Como podemos ver, los resultados son bastante pobres, antes de cortar el data, voy a buscar más [modelos](https://scikit-learn.org/stable/modules/linear_model.html):


In [260]:
models2 = { "elastic": ElasticNet (),
           "bayes": BayesianRidge(),
           "ard": ARDRegression(),
           "Tweedie": TweedieRegressor (),
           "Huber" : HuberRegressor(max_iter=100000),
           "theil": TheilSenRegressor (),
           "ransac": RANSACRegressor (),
          "lars": Lars ()
          }

In [440]:
probando_modelos (models2)

--------elastic--------
MSE:  0.3743261173264947
--------bayes--------
MSE:  0.051500817948219355
--------ard--------
MSE:  0.05167483955899185
--------Tweedie--------
MSE:  0.13294175049744572
--------Huber--------
MSE:  0.05722890512033469
--------theil--------
MSE:  0.05792434625302167
--------ransac--------
MSE:  0.05877877666908478
--------lars--------
MSE:  0.05150747922908036


Ninguno mejora, ni se acerca a los resultados obtenidos previamente. Asi que vamos a ver qué ocurre si modificamos el numero de features

In [265]:
data.corr()

,carat,cut,color,clarity,depth,table,x,y,z,price
carat,1.000000,-0.136512,-0.295110,-0.347222,0.029747,0.182453,0.974714,0.944353,0.948460,0.920292
cut,-0.136512,1.000000,0.024676,0.187792,-0.211912,-0.438306,-0.128336,-0.123047,-0.150653,-0.090436
color,-0.295110,0.024676,1.000000,-0.025310,-0.048786,-0.028571,-0.273433,-0.264744,-0.270077,-0.158279
clarity,-0.347222,0.187792,-0.025310,1.000000,-0.064298,-0.164575,-0.366545,-0.350738,-0.359719,-0.210824
depth,0.029747,-0.211912,-0.048786,-0.064298,1.000000,-0.291668,-0.023705,-0.028183,0.095489,0.003437
table,0.182453,-0.438306,-0.028571,-0.164575,-0.291668,1.000000,0.197420,0.184191,0.152496,0.159969
x,0.974714,-0.128336,-0.273433,-0.366545,-0.023705,0.197420,1.000000,0.967309,0.966396,0.957815
y,0.944353,-0.123047,-0.264744,-0.350738,-0.028183,0.184191,0.967309,1.000000,0.941970,0.928896
z,0.948460,-0.150653,-0.270077,-0.359719,0.095489,0.152496,0.966396,0.941970,1.000000,0.930774
price,0.920292,-0.090436,-0.158279,-0.210824,0.003437,0.159969,0.957815,0.928896,0.930774,1.000000


Empezamos con carat, que entiendo que es una feature en la que inciden directamente otras 3 (x, y z)

In [266]:
datacopy= data.drop (["carat"], axis=1)

In [267]:
datacopy

,cut,color,clarity,depth,table,x,y,z,price
0,4,4,3.0,62.7,58.0,6.36,6.31,3.97,8.754
1,2,4,2.5,64.3,57.0,5.31,5.38,3.43,7.492
2,4,3,1.5,59.4,59.0,7.55,7.60,4.50,9.010
3,3,1,3.0,63.5,58.0,4.27,4.24,2.70,6.227
4,3,5,1.5,62.8,57.0,6.37,6.44,4.02,8.314
...,...,...,...,...,...,...,...,...,...
40450,5,6,2.5,62.5,55.0,5.08,5.13,3.19,7.412
40451,5,3,2.5,62.1,54.0,5.96,5.99,3.71,8.016
40452,4,4,1.5,58.7,59.0,7.51,7.45,4.39,9.070
40453,5,2,2.5,61.6,54.0,7.38,7.42,4.57,9.199


In [282]:
columnas_x2 = [a for a in (list(datacopy.columns)) if a != "price"]
X2 = datacopy[columnas_x2]
y2= datacopy.price

In [285]:
gs3 = GridSearchCV(
    estimator=GradientBoostingRegressor(criterion="mse"),
    param_grid={
        "n_estimators": [700,800,900,1000,1100,1200,1300,1400],
        "learning_rate":[0.2,0.25,]
    },
    cv=4,
    verbose=3,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

In [318]:
gs3.fit (X2, y2 )

Fitting 4 folds for each of 27 candidates, totalling 108 fits
[CV] learning_rate=0.2, n_estimators=700 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.2, n_estimators=700, score=(train=-0.007, test=-0.009), total=  14.9s
[CV] learning_rate=0.2, n_estimators=700 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.0s remaining:    0.0s


KeyboardInterrupt: 

In [425]:
resultados_gridGBR (gs3)

,param_n_estimators,param_learning_rate,mean_test_score,mean_train_score
15,1300,0.25,-0.008412,-0.005649
16,1400,0.25,-0.008413,-0.005544
14,1200,0.25,-0.008429,-0.005756
17,1500,0.25,-0.008430,-0.005449
12,1000,0.25,-0.008438,-0.005976
13,1100,0.25,-0.008460,-0.005861
10,800,0.25,-0.008502,-0.006260
11,900,0.25,-0.008506,-0.006110
9,700,0.25,-0.008539,-0.006438
5,1200,0.2,-0.008541,-0.006005


Los resultados no son mejores, y esto tarda demasiado, primero hay que hacer pruebas a ver con qué features podemos jugar: (realizadas en el punto 1 del jupyter de apoyo)



Después de explorar las mejores combinaciones y sabiendo lo que tarda el gridsearch, vamos a preparar 6 pruebas más, 3 en este notebook y 3 en el otro (Punto 2) para poder ver los resultados mas tarde:

Aquí veremos nuestro train sin table, sin depth ni cut y sin depth ni table

In [332]:
sindepthtable= data.drop (["depth", "table"], axis=1)
sindepthcut= data.drop (["depth", "cut"], axis=1)
sintable= data.drop (["depth", "cut"], axis=1)

In [333]:
columnas_x = [a for a in (list(sindepthtable.columns)) if a != "price"]
X3 = sindepthtable[columnas_x]
y3= sindepthtable.price

In [338]:
columnas_x = [a for a in (list(sindepthcut.columns)) if a != "price"]
X4=sindepthcut[columnas_x]
y4= sindepthcut.price

In [339]:
columnas_x = [a for a in (list(sintable.columns)) if a != "price"]
X5=sintable[columnas_x]
y5=sintable.price

In [336]:
gs4 = GridSearchCV(
    estimator=GradientBoostingRegressor(criterion="mse"),
    param_grid={
        "n_estimators": [700,800,900,1000,1100,1200,1300,1400],
        "learning_rate":[0.22, 0.25]
    },
    cv=5,
    verbose=3,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

In [360]:
# es el mismo gridsearch para todos, lo lógico hubiera sido hacer algo como "sintable=gs3.fit (X3, y3)" 
# y no crear 3 gridsearch iguales, pero puse esto antes de ausentarme, y no sabía exactamente que iba a ser ese "sintable" al 100%

In [337]:
gs4.fit (X3,y3)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  15.3s
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.4s remaining:    0.0s


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  14.8s
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   30.4s remaining:    0.0s


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  14.8s
[CV] learning_rate=0.22, n_estimators=700 ............................
[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  15.0s
[CV] learning_rate=0.22, n_estimators=700 ............................
[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  15.4s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_rate=0.22, n_estimators=800, score=(train=-0.006, test=-0.008), total=  17.7s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_rate=0.22, n_estimators=800, score=(train=-0.006, test=-0.008), total=  17.0s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_rate=0.22, n_estimators=800, score=(train=-0.006, test=-0.008), total=  16.6s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_r

[CV]  learning_rate=0.25, n_estimators=900, score=(train=-0.006, test=-0.008), total=  19.6s
[CV] learning_rate=0.25, n_estimators=900 ............................
[CV]  learning_rate=0.25, n_estimators=900, score=(train=-0.006, test=-0.008), total=  19.6s
[CV] learning_rate=0.25, n_estimators=900 ............................
[CV]  learning_rate=0.25, n_estimators=900, score=(train=-0.006, test=-0.008), total=  19.0s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learning_rate=0.25, n_estimators=1000, score=(train=-0.006, test=-0.008), total=  21.4s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learning_rate=0.25, n_estimators=1000, score=(train=-0.006, test=-0.008), total=  21.4s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learning_rate=0.25, n_estimators=1000, score=(train=-0.006, test=-0.008), total=  21.1s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learnin

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 30.3min finished


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(criterion='mse'),
             param_grid={'learning_rate': [0.22, 0.25],
                         'n_estimators': [700, 800, 900, 1000, 1100, 1200, 1300,
                                          1400]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=3)

In [424]:
resultados_gridGBR (gs4)

,param_n_estimators,param_learning_rate,mean_test_score,mean_train_score
7,1400,0.22,-0.007723,-0.005507
5,1200,0.22,-0.007730,-0.005693
6,1300,0.22,-0.007734,-0.005595
3,1000,0.22,-0.007739,-0.005905
4,1100,0.22,-0.007750,-0.005788
12,1100,0.25,-0.007751,-0.005646
14,1300,0.25,-0.007755,-0.005448
11,1000,0.25,-0.007762,-0.005753
2,900,0.22,-0.007768,-0.006016
15,1400,0.25,-0.007768,-0.005361


In [343]:
columnas_x = [a for a in (list(sindepthcut.columns)) if a != "price"]
X4=sindepthcut[columnas_x]
y4= sindepthcut.price

In [344]:
columnas_x = [a for a in (list(sintable.columns)) if a != "price"]
X5=sintable[columnas_x]
y5=sintable.price

In [346]:
gs5 = GridSearchCV(
    estimator=GradientBoostingRegressor(criterion="mse"),
    param_grid={
        "n_estimators": [700,800,900,1000,1100,1200,1300,1400],
        "learning_rate":[0.22, 0.25]
    },
    cv=5,
    verbose=3,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

In [347]:
gs5.fit (X4, y4)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  17.7s
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.8s remaining:    0.0s


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  17.6s
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   35.7s remaining:    0.0s


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  18.1s
[CV] learning_rate=0.22, n_estimators=700 ............................
[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  17.2s
[CV] learning_rate=0.22, n_estimators=700 ............................
[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  19.4s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_rate=0.22, n_estimators=800, score=(train=-0.006, test=-0.008), total=  20.4s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_rate=0.22, n_estimators=800, score=(train=-0.006, test=-0.008), total=  20.8s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_rate=0.22, n_estimators=800, score=(train=-0.006, test=-0.008), total=  20.4s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_r

[CV]  learning_rate=0.25, n_estimators=900, score=(train=-0.006, test=-0.008), total=  22.0s
[CV] learning_rate=0.25, n_estimators=900 ............................
[CV]  learning_rate=0.25, n_estimators=900, score=(train=-0.006, test=-0.008), total=  22.4s
[CV] learning_rate=0.25, n_estimators=900 ............................
[CV]  learning_rate=0.25, n_estimators=900, score=(train=-0.006, test=-0.008), total=  22.1s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learning_rate=0.25, n_estimators=1000, score=(train=-0.006, test=-0.008), total=  24.7s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learning_rate=0.25, n_estimators=1000, score=(train=-0.006, test=-0.008), total=  24.5s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learning_rate=0.25, n_estimators=1000, score=(train=-0.006, test=-0.008), total=  24.5s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learnin

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 35.7min finished


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(criterion='mse'),
             param_grid={'learning_rate': [0.22, 0.25],
                         'n_estimators': [700, 800, 900, 1000, 1100, 1200, 1300,
                                          1400]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=3)

In [423]:
resultados_gridGBR (gs5)

,param_n_estimators,param_learning_rate,mean_test_score,mean_train_score
13,1200,0.25,-0.007870,-0.005644
14,1300,0.25,-0.007875,-0.005546
15,1400,0.25,-0.007880,-0.005452
7,1400,0.22,-0.007884,-0.005637
12,1100,0.25,-0.007895,-0.005753
6,1300,0.22,-0.007903,-0.005726
5,1200,0.22,-0.007906,-0.005823
11,1000,0.25,-0.007915,-0.005878
4,1100,0.22,-0.007925,-0.005928
3,1000,0.22,-0.007926,-0.006040


In [350]:
gs6 = GridSearchCV(
    estimator=GradientBoostingRegressor(criterion="mse"),
    param_grid={
        "n_estimators": [700,800,900,1000,1100,1200,1300,1400],
        "learning_rate":[0.22, 0.25]
    },
    cv=5,
    verbose=3,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

In [351]:
gs6.fit (X5, y5)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  17.3s
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.5s remaining:    0.0s


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  17.5s
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   35.2s remaining:    0.0s


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  17.1s
[CV] learning_rate=0.22, n_estimators=700 ............................
[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  17.5s
[CV] learning_rate=0.22, n_estimators=700 ............................
[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  17.3s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_rate=0.22, n_estimators=800, score=(train=-0.006, test=-0.008), total=  19.6s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_rate=0.22, n_estimators=800, score=(train=-0.006, test=-0.008), total=  19.6s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_rate=0.22, n_estimators=800, score=(train=-0.006, test=-0.008), total=  19.8s
[CV] learning_rate=0.22, n_estimators=800 ............................
[CV]  learning_r

[CV]  learning_rate=0.25, n_estimators=900, score=(train=-0.006, test=-0.008), total=  22.0s
[CV] learning_rate=0.25, n_estimators=900 ............................
[CV]  learning_rate=0.25, n_estimators=900, score=(train=-0.006, test=-0.008), total=  22.2s
[CV] learning_rate=0.25, n_estimators=900 ............................
[CV]  learning_rate=0.25, n_estimators=900, score=(train=-0.006, test=-0.008), total=  22.4s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learning_rate=0.25, n_estimators=1000, score=(train=-0.006, test=-0.008), total=  25.2s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learning_rate=0.25, n_estimators=1000, score=(train=-0.006, test=-0.008), total=  25.0s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learning_rate=0.25, n_estimators=1000, score=(train=-0.006, test=-0.008), total=  25.2s
[CV] learning_rate=0.25, n_estimators=1000 ...........................
[CV]  learnin

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 35.1min finished


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(criterion='mse'),
             param_grid={'learning_rate': [0.22, 0.25],
                         'n_estimators': [700, 800, 900, 1000, 1100, 1200, 1300,
                                          1400]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=3)

In [422]:
resultados_gridGBR (gs6)

,param_n_estimators,param_learning_rate,mean_test_score,mean_train_score
13,1200,0.25,-0.007859,-0.005644
14,1300,0.25,-0.007869,-0.005546
15,1400,0.25,-0.007881,-0.005452
7,1400,0.22,-0.007882,-0.005637
12,1100,0.25,-0.007886,-0.005753
6,1300,0.22,-0.007887,-0.005726
5,1200,0.22,-0.007902,-0.005823
4,1100,0.22,-0.007911,-0.005928
11,1000,0.25,-0.007922,-0.005878
3,1000,0.22,-0.007926,-0.006040


In [357]:
modeldepthtable = gs4.best_estimator_
modeldepthtable

GradientBoostingRegressor(criterion='mse', learning_rate=0.22,
                          n_estimators=1400)

In [359]:
modeldepthcut = gs5.best_estimator_
modeldepthcut

GradientBoostingRegressor(criterion='mse', learning_rate=0.25,
                          n_estimators=1200)

In [358]:
modeltable = gs6.best_estimator_
modeltable

GradientBoostingRegressor(criterion='mse', learning_rate=0.25,
                          n_estimators=1200)

En este punto, me quedaré con las 5 mejores puntuaciones, y haré 5 nuevas previsiones, antes de hacer la submision, haré un gridsearch con el data entero. El motivo de esto es que las conclusiones obtenidas para la primera submision fueron sacadas casi manuealmente, y con el gridsearch es posible obtener conclusiones más certeras.

Haré todo en este jupyter, pero las conclusiones son sobre ambos jupyters. Primero las ordenamos:
* modeldepth= -0.007608
* modeldepthtable= -0.007723
* modelcut= -0.007829
* modeltable= -0.007859
* modeldepthcut= -0.007865
---------
* modeldepthcut= -0.007870


Visto esto, la única razón de descartar el modelo es la limitación de submissions, no me parece que la diferencia sea suficientemente amplia como para no querer probar también el sexto modelo

Primero preparamos los datas iniciales, tengo que crear los datas sin las features hechos en el otro jupyter, y luego modificar los datas de los test:

In [482]:
sindepth= data.drop (["depth",], axis=1)
sincut= data.drop (["cut"], axis=1)

In [483]:
tsindepthtable= datat.drop (["depth", "table"], axis=1)
tsindepthcut= datat.drop (["depth", "cut"], axis=1)
tsintable= datat.drop (["depth", "cut"], axis=1)
tsindepth= datat.drop (["depth",], axis=1)
tsincut= datat.drop (["cut"], axis=1)

Sin Cut:
* definir modelo con resultados de best estimator
* entrenar modelo con X,y, siendo X,y el data de entrenamiento.
* hacer predict sobre t.
* poner el y pred como precio en t.
* renombrar index y eliminar resto de columnas
* salvar csv


In [484]:
gradient_model3 = GradientBoostingRegressor (n_estimators=1400, learning_rate=0.25, criterion="mse")

In [485]:
columnas_x = [a for a in (list(sincut.columns)) if a != "price"]
X6=sincut[columnas_x]
y6= sincut.price

In [486]:
gradient_model3.fit (X6, y6)

GradientBoostingRegressor(criterion='mse', learning_rate=0.25,
                          n_estimators=1400)

In [487]:
y_pred=gradient_model3.predict (tsincut)

In [488]:
preparando_submision (tsincut, "gradientsincut")

Sin depth

In [489]:
gradient_model4 = GradientBoostingRegressor (n_estimators=1400, learning_rate=0.22, criterion="mse")

In [490]:
columnas_x = [a for a in (list(sindepth.columns)) if a != "price"]
X7=sindepth[columnas_x]
y7= sindepth.price

In [491]:
gradient_model4.fit (X7, y7)

GradientBoostingRegressor(criterion='mse', learning_rate=0.22,
                          n_estimators=1400)

In [492]:
y_pred = gradient_model4.predict (tsindepth)

In [493]:
preparando_submision (tsindepth, "gradientsindepth")

Sin Table

In [494]:
gradient_model5 = GradientBoostingRegressor (n_estimators=1200, learning_rate=0.25, criterion="mse")

In [495]:
gradient_model5.fit (X5, y5)

GradientBoostingRegressor(criterion='mse', learning_rate=0.25,
                          n_estimators=1200)

In [496]:
y_pred = gradient_model5.predict (tsintable)

In [499]:
preparando_submision (tsintable, "gradientsintable")

Sin depth ni cut

In [500]:
gradient_model6 = GradientBoostingRegressor (n_estimators=1200, learning_rate=0.25, criterion="mse")

In [501]:
gradient_model6.fit (X4, y4)

GradientBoostingRegressor(criterion='mse', learning_rate=0.25,
                          n_estimators=1200)

In [502]:
y_pred = gradient_model6.predict (tsindepthcut)

In [503]:
preparando_submision (tsindepthcut, "gradientsindepthcut")

Sin depth ni table

In [504]:
gradient_model7 = GradientBoostingRegressor (n_estimators=1400, learning_rate=0.22, criterion="mse")

In [505]:
gradient_model7.fit (X3, y3)

GradientBoostingRegressor(criterion='mse', learning_rate=0.22,
                          n_estimators=1400)

In [506]:
y_pred = gradient_model7.predict (tsindepthtable)

In [507]:
preparando_submision (tsindepthtable, "gradientsindepthtable")

Y con esto terminaríamos, pero como dije antes, le quiero hacer un gridsearch al train completo, vamos a ello:

In [415]:
gsfinal = GridSearchCV(
    estimator=GradientBoostingRegressor(criterion="mse"),
    param_grid={
        "n_estimators": [700,800,900,1000,1100,1200,1300,1400, 1500,1600,1700,1800],
        "learning_rate":[0.22, 0.23, 0.24, 0.25]
    },
    cv=5,
    verbose=3,
    scoring="neg_mean_squared_error",
    return_train_score=True
)

In [416]:
gsfinal.fit (X, y) # Y ahora es cuando nos podemos ir dar un paseo, volcer, hacer la comida, echar la siesta...

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  18.2s
[CV] learning_rate=0.22, n_estimators=700 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.3s remaining:    0.0s


[CV]  learning_rate=0.22, n_estimators=700, score=(train=-0.006, test=-0.008), total=  18.0s
[CV] learning_rate=0.22, n_estimators=700 ............................


...

[CV]  learning_rate=0.23, n_estimators=1500, score=(train=-0.005, test=-0.007), total=  38.0s
[CV] learning_rate=0.23, n_estimators=1500 ...........................
[CV]  learning_rate=0.23, n_estimators=1500, score=(train=-0.005, test=-0.008), total=  37.9s
[CV] learning_rate=0.23, n_estimators=1500 ...........................
[CV]  learning_rate=0.23, n_estimators=1500, score=(train=-0.005, test=-0.008), total=  37.8s
[CV] learning_rate=0.23, n_estimators=1600 ...........................
[CV]  learning_rate=0.23, n_estimators=1600, score=(train=-0.005, test=-0.008), total=  42.9s
[CV] learning_rate=0.23, n_estimators=1600 ...........................
[CV]  learning_rate=0.23, n_estimators=1600, score=(train=-0.005, test=-0.007), total=  42.2s
[CV] learning_rate=0.23, n_estimators=1600 ...........................
[CV]  learning_rate=0.23, n_estimators=1600, score=(train=-0.005, test=-0.007), total=  41.0s
[CV] learning_rate=0.23, n_estimators=1600 ...........................
[CV]  lear

[CV]  learning_rate=0.24, n_estimators=1300, score=(train=-0.005, test=-0.007), total=  33.7s
[CV] learning_rate=0.24, n_estimators=1300 ...........................
[CV]  learning_rate=0.24, n_estimators=1300, score=(train=-0.005, test=-0.008), total=  36.6s
[CV] learning_rate=0.24, n_estimators=1300 ...........................
[CV]  learning_rate=0.24, n_estimators=1300, score=(train=-0.005, test=-0.008), total=  33.9s
[CV] learning_rate=0.24, n_estimators=1400 ...........................
[CV]  learning_rate=0.24, n_estimators=1400, score=(train=-0.005, test=-0.008), total=  36.4s
[CV] learning_rate=0.24, n_estimators=1400 ...........................
[CV]  learning_rate=0.24, n_estimators=1400, score=(train=-0.005, test=-0.008), total=  36.4s
[CV] learning_rate=0.24, n_estimators=1400 ...........................
[CV]  learning_rate=0.24, n_estimators=1400, score=(train=-0.005, test=-0.008), total=  36.5s
[CV] learning_rate=0.24, n_estimators=1400 ...........................
[CV]  lear

[CV]  learning_rate=0.25, n_estimators=1100, score=(train=-0.006, test=-0.008), total=  28.7s
[CV] learning_rate=0.25, n_estimators=1100 ...........................
[CV]  learning_rate=0.25, n_estimators=1100, score=(train=-0.006, test=-0.008), total=  28.7s
[CV] learning_rate=0.25, n_estimators=1100 ...........................
[CV]  learning_rate=0.25, n_estimators=1100, score=(train=-0.005, test=-0.008), total=  28.6s
[CV] learning_rate=0.25, n_estimators=1200 ...........................
[CV]  learning_rate=0.25, n_estimators=1200, score=(train=-0.005, test=-0.008), total=  31.1s
[CV] learning_rate=0.25, n_estimators=1200 ...........................
[CV]  learning_rate=0.25, n_estimators=1200, score=(train=-0.005, test=-0.007), total=  30.7s
[CV] learning_rate=0.25, n_estimators=1200 ...........................
[CV]  learning_rate=0.25, n_estimators=1200, score=(train=-0.005, test=-0.008), total=  31.8s
[CV] learning_rate=0.25, n_estimators=1200 ...........................
[CV]  lear

[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 131.6min finished


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(criterion='mse'),
             param_grid={'learning_rate': [0.22, 0.23, 0.24, 0.25],
                         'n_estimators': [700, 800, 900, 1000, 1100, 1200, 1300,
                                          1400, 1500, 1600, 1700, 1800]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=3)

In [421]:
resultados_gridGBR (gsfinal)

,param_n_estimators,param_learning_rate,mean_test_score,mean_train_score
9,1600,0.22,-0.007514,-0.005202
8,1500,0.22,-0.007519,-0.005284
11,1800,0.22,-0.007528,-0.005060
10,1700,0.22,-0.007528,-0.005128
7,1400,0.22,-0.007528,-0.005371
5,1200,0.22,-0.007543,-0.005560
6,1300,0.22,-0.007546,-0.005467
4,1100,0.22,-0.007549,-0.005662
23,1800,0.23,-0.007557,-0.004994
19,1400,0.23,-0.007564,-0.005304


Los resultados son mejores que los anteriores, asi que finalmente haré una submision mas con estos nuevos resultados, y las otras 3 que me quedan con las anteriores

In [508]:
gsmodel= GradientBoostingRegressor(criterion="mse", n_estimators=1600, learning_rate=0.22)

In [509]:
gsmodel.fit (X,y)

GradientBoostingRegressor(criterion='mse', learning_rate=0.22,
                          n_estimators=1600)

In [511]:
y_pred=gsmodel.predict (datat)

In [512]:
preparando_submision (datat, "gradient3")

Para la ultima submission voy a volver a pasar la función que he creado para probar las features, pero le voy a aumentar el n_estimators. Realizado en el otro jupyter. Es el punto 3

No ha terminado de ejecutarse, pero no parece que me vaya a mejorar los resultados mucho, así que por hacer algo distinto, voy a pasarle la función para probar features, pero le voy a modificar la salida, en lugar de mostrar el mse, le vamos a pedir el error absoluto. Con esto lo que pretendo es combinar de alguna manera varios modelos para crear una previsión con la media, pero para ello intentaré saber primero si el error es positivo o negativo.

In [517]:
gradient_model2=  GradientBoostingRegressor (n_estimators=1500, learning_rate=0.25, criterion="mse")

In [518]:
prueba_features3 (data, gradient_model2, "price" )

carat
MSE:  0.011238635175432343
CROSSMSE -0.06346891164999417
cut
MSE:  0.011060973701474266
CROSSMSE -0.06351186260842807
color
MSE:  0.010690095551482457
CROSSMSE -0.06352008435481252
clarity
MSE:  0.011225400363655722
CROSSMSE -0.0635571673894569
depth
MSE:  0.010873740793754636
CROSSMSE -0.06344385558053973
table
MSE:  0.01121952791630236
CROSSMSE -0.06348513423122545
x
MSE:  0.01090369348457617
CROSSMSE -0.06373934360940424
y
MSE:  0.010435088506519804
CROSSMSE -0.06405368428921704
z
MSE:  0.0108747339994395
CROSSMSE -0.06312738621120191


No saco ninguna conclusión de esto, en cualquier caso voy a elegir las 2 mejores previsiones que he obtenido y sacar la media para la ultima submision.

In [531]:
data= pd.read_csv("../Data/train.csv")
datat=pd.read_csv ("../Data/test.csv")

In [532]:
categorizacion (data)
categorizacion(datat)

,carat,cut,color,clarity,depth,table,x,y,z
0,2.01,5,3,2.0,61.9,57.0,8.14,8.05,5.01
1,0.49,2,7,3.0,57.5,60.0,5.18,5.25,3.00
2,1.03,4,5,2.0,58.6,62.0,6.65,6.60,3.88
3,0.90,3,6,2.0,63.0,56.0,6.11,6.15,3.86
4,0.59,5,7,2.0,62.5,55.0,5.35,5.40,3.36
...,...,...,...,...,...,...,...,...,...
13480,0.41,2,5,1.0,63.8,57.0,4.72,4.69,3.00
13481,2.05,3,3,2.5,63.5,62.0,8.05,7.91,5.07
13482,2.22,4,2,2.5,60.5,59.0,8.41,8.37,5.08
13483,0.51,5,6,1.5,58.3,62.0,5.20,5.19,3.04


La más certera de las entregadas era un  GradientBoostingRegressor con criterion="mse" n_estimators=1600 learning_rate=0.22 y con todos los datos, la segunda la sacaré del que mejor resultados dé en el otro jupyter.


In [25]:
gsmodel= GradientBoostingRegressor(criterion="mse", n_estimators=1600, learning_rate=0.22)

In [26]:
gsmodel.fit (X,y)

GradientBoostingRegressor(criterion='mse', learning_rate=0.22,
                          n_estimators=1600)

In [27]:
y_pred=gsmodel.predict (datat)

In [28]:
datat["pred1"]= y_pred

In [45]:
entregafinal=datat[["pred1"]]

In [46]:
entregafinal.index.rename ("id", inplace=True)

La que mejor resultados ha dadado ha sido sin la "X", así que vamos a sacarla para promediar, debería hacer un gridsearch para buscar los mejores parámetros, pero no me va a dar tiempo:


In [15]:
sinx= data.drop (["x"], axis=1)
tsinx= datat.drop (["x"], axis=1)

In [17]:
columnas_x = [a for a in (list(sinx.columns)) if a != "price"]
X20=sinx[columnas_x]
y20=sinx.price

In [18]:
gsmodelx = GradientBoostingRegressor(criterion="mse", n_estimators=1600, learning_rate=0.22)

In [20]:
gsmodelx.fit (X20,y20)

GradientBoostingRegressor(criterion='mse', learning_rate=0.22,
                          n_estimators=1600)

In [22]:
y_pred2 = gsmodelx.predict (tsinx)

In [47]:
entregafinal["pred2"] = y_pred2

<ipython-input-47-fae9266e5d42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entregafinal["pred2"] = y_pred2


In [48]:
entregafinal["price"]= entregafinal.pred1/2 + entregafinal.pred2/2

<ipython-input-48-f630ff36416b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entregafinal["price"]= entregafinal.pred1/2 + entregafinal.pred2/2


In [50]:
entrega = entregafinal[["price"]]

In [53]:
entrega.to_csv ("../Data/ultimointento.csv")